Подключение базовых библиотек для обработки изображений

In [1]:
from IPython.display import Image
from imutils import paths
import numpy as np
import cv2
import os

Для работы с изображениями, найдем их гистограммы — характеристики распределения интенсивности изображения. Для этого можно воспользоваться следующим методом.

In [2]:
def extract_histogram(image, bins=(8, 8, 8)):
    hist = cv2.calcHist([image], [0, 1, 2], None, bins, [0, 256, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

Считаем изображения из корневой папки train и вычислим гистограмму каждого изображения. Кроме того, отделим метку каждого изображения. Обработка большого числа изображений в облачных сервисах занимает длительное время (1-5 минут).

In [3]:
imagePaths = sorted(list(paths.list_images('train')))
trainData = []
labels = []

for (i, imagePath) in enumerate(imagePaths):
    image = cv2.imread(imagePath, 1)
    label = imagePath.split(os.path.sep)[-1].split(".")[0]
    hist = extract_histogram(image)
    trainData.append(hist)
    labels.append(label)

Преобразуем метки в удобный формат 0 и 1. Cat заменяем на 1, Dog на 0.

In [4]:
Y = [1 if x == 'cat' else 0 for x in labels]

Обучаем первый базовый классификатор - бэггинг деревьев принятия решений (меняем значения возле комментариев на свои)

In [5]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion='entropy', #критерий разделения
                              min_samples_leaf=10, #минимальное число объектов в листе
                              max_leaf_nodes=20, #максимальное число листьев
                              random_state=336) #random_state
bagging = BaggingClassifier(tree, #базовый алгоритм
                            n_estimators=21, #количество деревьев
                            random_state=336) #random_state
bagging.fit(trainData, Y)

BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                        max_leaf_nodes=20,
                                                        min_samples_leaf=10,
                                                        random_state=336),
                  n_estimators=21, random_state=336)

Теперь обучим модель почти-разделяющий гиперплосоксти (меняем значения возле комментариев на свои).

In [6]:
from sklearn.svm import LinearSVC

svm = LinearSVC(random_state = 336, C = 1.51) #random_state и C
svm.fit(trainData, Y)

LinearSVC(C=1.51, random_state=336)

И наконец обучаем третий базовый алгоритм — случайный лес

In [7]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=21, #количество деревьев
                             criterion='entropy', #критерий разделения
                              min_samples_leaf=10, #минимальное число объектов в листе
                              max_leaf_nodes=20, #максимальное число листьев
                              random_state=336)
forest.fit(trainData, Y)

RandomForestClassifier(criterion='entropy', max_leaf_nodes=20,
                       min_samples_leaf=10, n_estimators=21, random_state=336)

Задаем в качестве решающего метаалгоритма логистическую регрессию:

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='lbfgs', random_state=336) #solver и random_state

Обучаем метаалгоритм

In [9]:
from sklearn.ensemble import StackingClassifier

base_estimators = [('SVM', svm), ('Bagging DT', bagging), ('DecisionForest', forest)]
sclf = StackingClassifier(estimators=base_estimators, final_estimator=lr, cv=2)
sclf.fit(trainData, Y)

StackingClassifier(cv=2,
                   estimators=[('SVM', LinearSVC(C=1.51, random_state=336)),
                               ('Bagging DT',
                                BaggingClassifier(base_estimator=DecisionTreeClassifier(criterion='entropy',
                                                                                        max_leaf_nodes=20,
                                                                                        min_samples_leaf=10,
                                                                                        random_state=336),
                                                  n_estimators=21,
                                                  random_state=336)),
                               ('DecisionForest',
                                RandomForestClassifier(criterion='entropy',
                                                       max_leaf_nodes=20,
                                                       min_samples_leaf=10,
       

Оценка метамодели. Доля правильной классификации (Accuracy):

In [10]:
sclf.score(trainData, Y)

0.852

Выполняем предсказания для своих изображений.

In [20]:
images = [cv2.imread('test/dog.1038.jpg'), cv2.imread('test/cat.1012.jpg'), cv2.imread('test/cat.1022.jpg'), cv2.imread('test/dog.1021.jpg')] #заменяем названия файлов
#в выводе: первое число - вероятность отнесения изображения к собаке (0), а второе к кошке (1)
for picture in images:
    histt = extract_histogram(picture)
    histt2 = histt.reshape(1, -1)
    prediction = sclf.predict(histt2)
    print(sclf.predict_proba(histt2), end="\n")

[[0.55926714 0.44073286]]
[[0.77809232 0.22190768]]
[[0.53363117 0.46636883]]
[[0.49613521 0.50386479]]
